<h1>Simple application specific chat bot</h1>
<p>inspired by Tech with Tim(https://www.youtube.com/watch?v=wypVcNIH6D4)<br>
Difference between the original tutorial and this example are:<br>
1. NN model created using TensorFlow 2.0 'layers' and 'model' API instead of TFlearn<br>
2. Include deployment example of the app using Flask<br>
3. Tryed out different bag of words scoring (https://towardsdatascience.com/3-basic-approaches-in-bag-of-words-which-are-better-than-word-embeddings-c2cbc7398016)
</p>
<h2>Disclaimer</h2>
<p>NN model here in overfitted to the dataset and further callibration is needed with more data.<br>
This is just an example of how one can create simple chatbot and deploy it</p>

In [1]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
# import tflearn
import random
import json

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\taiseii\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
with open('intents.json') as file:
    data = json.load(file)
words = []
labels = []
docs_x = []
docs_y = []

In [3]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

words = [stemmer.stem(w.lower()) for w in words if w != "?" and w != "!"]
words = sorted(list(set(words)))

labels = sorted(labels)

stop_words = ['you', 'is', 'your','a']
#remove stop words
words = [w for w in words if w not in stop_words]

In [4]:
words

['ag',
 'am',
 'any',
 'anyon',
 'ar',
 'ass',
 'book',
 'cal',
 'colledg',
 'cya',
 'day',
 'dick',
 'did',
 'die',
 'do',
 'dur',
 'favorit',
 'fre',
 'fuck',
 'go',
 'good',
 'goodby',
 'hav',
 'hello',
 'hi',
 'hobby',
 'hol',
 'how',
 'i',
 'in',
 'lat',
 'learn',
 'leav',
 'lik',
 'list',
 'mus',
 'my',
 'nam',
 'off',
 'old',
 'or',
 'piss',
 'read',
 'see',
 'shit',
 'should',
 'skil',
 'someth',
 'song',
 'study',
 'suck',
 'taisei',
 'that',
 'ther',
 'tim',
 'to',
 'univers',
 'up',
 'want',
 'what',
 'when',
 'yo']

<h2>Using binary representation as feature</h2>

In [5]:
#binary representation of the words
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)



<h2>Using TF-IDF(Term Frequency-Inverse Document Frequency)</h2>

In [6]:
space = ' '
corpus = ''
for item in list(chain(*docs_x)):
    corpus += str(item) + str(space)

NameError: name 'chain' is not defined

In [7]:
corpus=[corpus,]

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer()
tfidf_count_occurs = tfidf_vec.fit_transform(corpus)
tfidf_count_occur_df = pd.DataFrame(
    (count, word) for word, count in zip(
    tfidf_count_occurs.toarray().tolist()[0],   
    tfidf_vec.get_feature_names()))
tfidf_count_occur_df.columns = ['Word', 'Count']
tfidf_count_occur_df.sort_values('Count', ascending=False, inplace=True)
tfidf_count_occur_df

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [9]:
training_tfidf = []
count = tfidf_count_occur_df[ tfidf_count_occur_df['Word']=='you']['Count']


for x, doc in enumerate(docs_x):
    bag = []
    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        # print(w)
        tfidf = tfidf_count_occur_df[ tfidf_count_occur_df['Word']==str(w)]['Count']
        try:
            bag.append(float(tfidf))
        except:
            bag.append(0)

    training_tfidf.append(bag)
    


NameError: name 'tfidf_count_occur_df' is not defined

In [10]:
training_tfidf = np.array(training_tfidf)

In [11]:
print(training_tfidf[0])
print(training_tfidf[1])

IndexError: index 0 is out of bounds for axis 0 with size 0

In [12]:
output = [o.index(1) for o in output]

In [13]:
output

[3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 6,
 6,
 6,
 9,
 9,
 9,
 4,
 4,
 4,
 4,
 1,
 1,
 1,
 7,
 7,
 7,
 5,
 5,
 8,
 8,
 8,
 8,
 8,
 8]

In [14]:
#categorilize the tag based on the input
training = np.array(training)
output = np.array(output)

In [15]:
len(words)

62

In [16]:
len(training[0])

62

In [17]:
len(training[0])

62

<h2>Building NN</h2>

In [18]:
# building NN here
num_classes = 10
num_features = len(training[0])
# Training parameters.
learning_rate = 0.1
training_steps = 2000
batch_size = 10
display_step = 100
# Network parameters.
n_hidden_1 = 8 # 1st layer number of neurons.
n_hidden_2 = 16 # 2nd layer number of neurons.
n_hidden_3 = 32
n_hidden_4 = 64

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(training, output,
                                                    test_size=0.3,
                                                    random_state=42)
#or just overfit it on purpose... let's try both

In [20]:
import tensorflow as tf
from tensorflow.keras import Model, layers

C:\Users\taiseii\Anaconda3\envs\tensor_flow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [22]:
# Create TF Model.
class NeuralNet(Model):
    # Set layers.
    def __init__(self):
        super(NeuralNet, self).__init__()
        # First fully-connected hidden layer.
        self.fc1 = layers.Dense(n_hidden_1, activation=tf.nn.relu)
        # First fully-connected hidden layer.
        self.fc2 = layers.Dense(n_hidden_2, activation=tf.nn.relu)
        # Second fully-connecter hidden layer.
        self.fc3 = layers.Dense(n_hidden_3, activation=tf.nn.relu)
        self.fc4 = layers.Dense(n_hidden_4, activation=tf.nn.relu)
        self.out = layers.Dense(num_classes, activation=tf.nn.softmax)

    # Set forward pass.
    def call(self, x, is_training=False):
        x = self.fc1(x)
        x = self.out(x)
        if not is_training:
            # tf cross entropy expect logits without softmax, so only
            # apply softmax when not training.
            x = tf.nn.softmax(x)
        return x

# Build neural network model.
neural_net = NeuralNet()

In [23]:
# Cross-Entropy Loss.
# Note that this will apply 'softmax' to the logits.
def cross_entropy_loss(x, y):
    # Convert labels to int 64 for tf cross-entropy function.
    y = tf.cast(y, tf.int64)
    # Apply softmax to logits and compute cross-entropy.
#     print(y)
#     print(x)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    # Average loss across the batch.
    return tf.reduce_mean(loss)

# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.SGD(learning_rate)

In [24]:
# Optimization process. 
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        # Forward pass.
        pred = neural_net(x, is_training=True)
        # Compute loss.
#         print(pred)
#         print(y)
        loss = cross_entropy_loss(pred, y)
        
    # Variables to update, i.e. trainable variables.
    trainable_variables = neural_net.trainable_variables

    # Compute gradients.
    gradients = g.gradient(loss, trainable_variables)
    
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))

In [25]:
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
#     print(batch_y)
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0:
        pred = neural_net(batch_x, is_training=True)
        # print(batch_y)
        # print(pred)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 100, loss: 2.306026, accuracy: 0.000000
step: 200, loss: 2.297053, accuracy: 0.200000
step: 300, loss: 2.278134, accuracy: 0.500000
step: 400, loss: 2.311803, accuracy: 0.100000
step: 500, loss: 2.272221, accuracy: 0.300000
step: 600, loss: 2.178579, accuracy: 0.400000
step: 700, loss: 2.198611, accuracy: 0.300000
step: 800, loss: 2.213058, accuracy: 0.200000
step: 900, loss: 2.150500, accuracy: 0.300000
step: 1000, loss: 2.090604, accuracy: 0.300000
step: 1100, loss: 2.059720, accuracy: 0.400000
step: 1200, loss: 2.043406, accuracy: 0.600000
step: 1300, loss: 1.924723, accuracy: 0.700000
step: 1400, loss: 2.047070, accuracy: 0.400000
step: 1500, loss: 2.155752, accuracy: 0.300000
step: 1600, loss: 1.983356, accuracy: 0.400000
step: 1700, loss: 1.915985, accuracy: 0.500000
step: 1800, loss: 2.032921, accuracy: 0.400000
step: 1900, loss: 2.063818, accuracy: 0.400000
step: 2000, loss: 1.783082, accuracy: 0.800000


In [27]:
#save and load checkpoint!
# Save TF model.
neural_net.save_weights(filepath="./chat_bot_prototype.ckpt")
# Re-build neural network model with default values.
neural_net = NeuralNet()
# Test model performance.
pred = neural_net(batch_x)
print("accuracy: %f" % accuracy(pred, batch_y))
# Load saved weights.
neural_net.load_weights(filepath="./chat_bot_prototype.ckpt")
# Test that weights loaded correctly.
pred = neural_net(batch_x)
print("accuracy: %f" % accuracy(pred, batch_y))

accuracy: 0.000000
accuracy: 0.800000


In [187]:
# with tfidf
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data_tfidf.take(training_steps), 1):
    # Run the optimization to update W and b values.
#     print(batch_y)
    # print(batch_x[0])
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0:
        pred = neural_net(batch_x, is_training=True)
        # print(batch_y)
        # print(pred)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 100, loss: 2.285187, accuracy: 0.100000
step: 200, loss: 2.240403, accuracy: 0.200000
step: 300, loss: 2.222913, accuracy: 0.200000
step: 400, loss: 2.261308, accuracy: 0.200000
step: 500, loss: 2.261911, accuracy: 0.000000
step: 600, loss: 2.313378, accuracy: 0.100000
step: 700, loss: 2.340093, accuracy: 0.100000
step: 800, loss: 2.347739, accuracy: 0.100000
step: 900, loss: 2.204269, accuracy: 0.400000
step: 1000, loss: 2.279053, accuracy: 0.200000
step: 1100, loss: 2.233171, accuracy: 0.300000
step: 1200, loss: 2.270127, accuracy: 0.200000
step: 1300, loss: 2.269826, accuracy: 0.200000
step: 1400, loss: 2.325354, accuracy: 0.000000
step: 1500, loss: 2.226496, accuracy: 0.300000
step: 1600, loss: 2.320817, accuracy: 0.000000
step: 1700, loss: 2.282583, accuracy: 0.200000
step: 1800, loss: 2.325748, accuracy: 0.000000
step: 1900, loss: 2.160418, accuracy: 0.400000
step: 2000, loss: 2.238613, accuracy: 0.300000


<h2>Testing the bot</h2>

In [210]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return np.array([bag])

In [230]:
index = np.argmax( neural_net(bag_of_words('what to do', words)))
tag = labels[index]
for tg in data["intents"]:
    if tg['tag'] == tag:
        responses = tg['responses']

print(random.choice(responses))

I'm interested in machine learning


<h2>Loading from the checkpoint!</h2>

In [28]:
neural_net.load_weights(filepath="./chat_bot_chekpoints/chat_bot_prototype.ckpt")
pred = neural_net(batch_x)
print("accuracy: %f" % accuracy(pred, batch_y))

accuracy: 0.800000
